In [1]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 1
    PIN_MEMORY = True
    DATA_ORDER = "cw_ins"
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    
    # only flips
    MEAN_PIXEL = np.array([0.485, 0.456, 0.406],dtype=np.float32).reshape(1,1,-1)
    STD_PIXEL = np.array([0.229, 0.224, 0.225],dtype=np.float32).reshape(1,1,-1)
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    WIDTH = 224
    HEIGHT = 224
    CROP_SIZE = 224
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
train_image_dir = "/media/Data1/interns/aravind/train2017/"
# train_image_dir = "/media/Data1/interns/aravind/val2017/"
val_image_dir = "/media/Data1/interns/aravind/val2017/"
config = ProposalConfig()
model_dir = "./models/"
train_pickle = "/media/Data1/interns/aravind/re/data/train_cwid.pickle"
# train_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"
val_pickle = "/media/Data1/interns/aravind/re/data/val_cwid.pickle"

In [4]:
# with open(train_pickle,"rb") as train_ann:
#     train_cwid = pickle.load(train_ann)
with open(val_pickle,"rb") as val_ann:
    val_cwid = pickle.load(val_ann)

In [5]:
# train_loader = model_lib.get_loader(train_cwid,config,train_image_dir)
val_loader = model_lib.get_loader(val_cwid,config,val_image_dir)

In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image
net = model_lib.SimpleHGModel()

pretrained_dict = torch.load(model_dir+"gege2.pt")
net = model_lib.SimpleHGModel()
net_dict = net.state_dict()

pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

# net.vgg.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").vgg)
# net.classifier.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").classifier)
net = net.cuda()
with torch.no_grad():
    for i,data in enumerate(val_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = data
        print(config.CLASS_NAMES[np.argmax(batch_one_hot.numpy())])
        a,b,c,d= batch_images.numpy(), batch_impulses.numpy(),batch_gt_responses.numpy(),batch_bboxes.numpy()
        a = np.moveaxis(a,1,-1)
        b = np.moveaxis(b,1,-1)
        c = np.moveaxis(c,1,-1)
        d = np.moveaxis(d,1,-1)
        Image.fromarray(((a[0]*config.STD_PIXEL + config.MEAN_PIXEL)*255).astype(np.uint8),"RGB").show()
        Image.fromarray((b[0][:,:,0]*128).astype(np.uint8),"L").show()
        Image.fromarray((c[0][:,:,0]*128).astype(np.uint8),"L").show()
        Image.fromarray((d[0][:,:,0]*128).astype(np.uint8),"L").show()
        batch_images,batch_impulses,batch_gt_responses,batch_bboxes,batch_one_hot = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_bboxes.cuda(),batch_one_hot.cuda()
        pred_class,pred_masks = net([batch_images,batch_impulses])
#         print(pred_class)
#         pred_class = F.sigmoid(pred_class).squeeze()
        pred_class = F.softmax(pred_class,dim=-1).squeeze()
#         print(pred_class)
        maxs, indices = torch.topk(pred_class,5,-1)
#         print(pred_masks[0][-1].shape)
        pred_mask = pred_masks.squeeze()
        pred_masks = F.sigmoid(pred_masks)
#         pred_masks = F.threshold(pred_masks,0.5,0)
#         pred_masks = (pred_masks>0).float()
        pred_masks = pred_masks.squeeze().cpu().numpy()*255
        Image.fromarray(pred_masks.astype(np.uint8),"L").convert("RGB").show()

#         for i in range(10):
#             pred_mask = pred_masks[0][-1].squeeze()[i]
#             pred_mask = (F.sigmoid(pred_mask).clamp(0.5)*255).squeeze().cpu().numpy()
#             Image.fromarray(pred_mask.astype(np.uint8),"L").convert("RGB").show()
#             pred_mask = pred_masks[-1][-1].squeeze()[i]
#             pred_mask = (F.sigmoid(pred_mask).clamp(0.5)*255).squeeze().cpu().numpy()
#             Image.fromarray(pred_mask.astype(np.uint8),"L").convert("RGB").show()
#         Image.fromarray(pred_mask.astype(np.uint8),"L").convert("RGB").show()
#         Image.fromarray(pred_masks[1].astype(np.uint8),"L").convert("RGB").show()
#         Image.fromarray(pred_masks[2].astype(np.uint8),"L").convert("RGB").show()
#         Image.fromarray(pred_masks[3].astype(np.uint8),"L").convert("RGB").show()
#         print(maxs.shape,indices.shape)
        for i in range(5):
            print(maxs[i],indices[i],config.CLASS_NAMES[int(indices[i])])
#         print(batch_class_ids)
#         print(indices)
#         print("gt_class:",config.CLASS_NAMES[int(batch_class_ids[0])])
#         print("pred_class: ",pred_class[int(batch_class_ids[0])])
#         print(config.CLASS_NAMES[int(indices[0])])
        input()

couch
tensor(0.4408, device='cuda:0') tensor(1, device='cuda:0') person
tensor(0.1133, device='cuda:0') tensor(17, device='cuda:0') dog
tensor(1.00000e-02 *
       9.3412, device='cuda:0') tensor(16, device='cuda:0') cat
tensor(1.00000e-02 *
       5.9908, device='cuda:0') tensor(64, device='cuda:0') laptop
tensor(1.00000e-02 *
       4.5673, device='cuda:0') tensor(58, device='cuda:0') couch

cake
tensor(0.1713, device='cuda:0') tensor(56, device='cuda:0') cake
tensor(1.00000e-02 *
       9.3882, device='cuda:0') tensor(55, device='cuda:0') donut
tensor(1.00000e-02 *
       6.3139, device='cuda:0') tensor(16, device='cuda:0') cat
tensor(1.00000e-02 *
       5.8568, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-02 *
       3.6284, device='cuda:0') tensor(46, device='cuda:0') bowl

elephant
tensor(0.2844, device='cuda:0') tensor(21, device='cuda:0') elephant
tensor(0.1254, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       6.0339, devic

tensor(0.1131, device='cuda:0') tensor(59, device='cuda:0') potted plant
tensor(1.00000e-02 *
       9.5354, device='cuda:0') tensor(76, device='cuda:0') vase
tensor(1.00000e-02 *
       5.7561, device='cuda:0') tensor(41, device='cuda:0') wine glass
tensor(1.00000e-02 *
       5.3699, device='cuda:0') tensor(15, device='cuda:0') bird
tensor(1.00000e-02 *
       5.0844, device='cuda:0') tensor(46, device='cuda:0') bowl

potted plant
tensor(0.2132, device='cuda:0') tensor(59, device='cuda:0') potted plant
tensor(0.1038, device='cuda:0') tensor(46, device='cuda:0') bowl
tensor(1.00000e-02 *
       7.2845, device='cuda:0') tensor(42, device='cuda:0') cup
tensor(1.00000e-02 *
       7.0846, device='cuda:0') tensor(40, device='cuda:0') bottle
tensor(1.00000e-02 *
       6.7451, device='cuda:0') tensor(56, device='cuda:0') cake

baseball bat
tensor(0.1109, device='cuda:0') tensor(45, device='cuda:0') spoon
tensor(0.1010, device='cuda:0') tensor(44, device='cuda:0') knife
tensor(1.00000e-02 *


cup
tensor(1.00000e-02 *
       9.7573, device='cuda:0') tensor(37, device='cuda:0') skateboard
tensor(1.00000e-02 *
       9.4687, device='cuda:0') tensor(46, device='cuda:0') bowl
tensor(1.00000e-02 *
       7.6405, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-02 *
       5.9621, device='cuda:0') tensor(56, device='cuda:0') cake
tensor(1.00000e-02 *
       3.2991, device='cuda:0') tensor(36, device='cuda:0') baseball glove

BG
tensor(0.3235, device='cuda:0') tensor(16, device='cuda:0') cat
tensor(0.1644, device='cuda:0') tensor(56, device='cuda:0') cake
tensor(0.1215, device='cuda:0') tensor(17, device='cuda:0') dog
tensor(1.00000e-02 *
       7.8713, device='cuda:0') tensor(78, device='cuda:0') teddy bear
tensor(1.00000e-02 *
       6.0793, device='cuda:0') tensor(49, device='cuda:0') sandwich

car
tensor(0.1676, device='cuda:0') tensor(74, device='cuda:0') book
tensor(0.1337, device='cuda:0') tensor(63, device='cuda:0') tv
tensor(1.00000e-02 *
       7.8073


potted plant
tensor(0.4476, device='cuda:0') tensor(59, device='cuda:0') potted plant
tensor(0.2401, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       4.6943, device='cuda:0') tensor(40, device='cuda:0') bottle
tensor(1.00000e-02 *
       2.2398, device='cuda:0') tensor(76, device='cuda:0') vase
tensor(1.00000e-02 *
       1.8335, device='cuda:0') tensor(41, device='cuda:0') wine glass

hair drier
tensor(0.2068, device='cuda:0') tensor(75, device='cuda:0') clock
tensor(1.00000e-02 *
       5.5202, device='cuda:0') tensor(42, device='cuda:0') cup
tensor(1.00000e-02 *
       4.4540, device='cuda:0') tensor(66, device='cuda:0') remote
tensor(1.00000e-02 *
       4.2850, device='cuda:0') tensor(46, device='cuda:0') bowl
tensor(1.00000e-02 *
       3.8459, device='cuda:0') tensor(3, device='cuda:0') car

kite
tensor(0.3430, device='cuda:0') tensor(38, device='cuda:0') surfboard
tensor(0.1336, device='cuda:0') tensor(30, device='cuda:0') frisbee
tensor(1.00000e-


zebra
tensor(0.2070, device='cuda:0') tensor(18, device='cuda:0') horse
tensor(0.1542, device='cuda:0') tensor(20, device='cuda:0') cow
tensor(0.1214, device='cuda:0') tensor(17, device='cuda:0') dog
tensor(1.00000e-02 *
       9.4895, device='cuda:0') tensor(23, device='cuda:0') zebra
tensor(1.00000e-02 *
       5.6299, device='cuda:0') tensor(1, device='cuda:0') person

motorcycle
tensor(0.1516, device='cuda:0') tensor(2, device='cuda:0') bicycle
tensor(0.1391, device='cuda:0') tensor(4, device='cuda:0') motorcycle
tensor(1.00000e-02 *
       5.1704, device='cuda:0') tensor(76, device='cuda:0') vase
tensor(1.00000e-02 *
       4.9236, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       4.7820, device='cuda:0') tensor(9, device='cuda:0') boat

laptop
tensor(0.1788, device='cuda:0') tensor(74, device='cuda:0') book
tensor(1.00000e-02 *
       7.5737, device='cuda:0') tensor(70, device='cuda:0') oven
tensor(1.00000e-02 *
       5.9117, device='cuda:0') tensor


giraffe
tensor(0.3516, device='cuda:0') tensor(1, device='cuda:0') person
tensor(1.00000e-02 *
       5.2314, device='cuda:0') tensor(21, device='cuda:0') elephant
tensor(1.00000e-02 *
       3.8196, device='cuda:0') tensor(27, device='cuda:0') handbag
tensor(1.00000e-02 *
       3.7758, device='cuda:0') tensor(40, device='cuda:0') bottle
tensor(1.00000e-02 *
       3.7021, device='cuda:0') tensor(18, device='cuda:0') horse

sink
tensor(0.1105, device='cuda:0') tensor(42, device='cuda:0') cup
tensor(0.1006, device='cuda:0') tensor(46, device='cuda:0') bowl
tensor(1.00000e-02 *
       6.6870, device='cuda:0') tensor(56, device='cuda:0') cake
tensor(1.00000e-02 *
       4.6895, device='cuda:0') tensor(17, device='cuda:0') dog
tensor(1.00000e-02 *
       4.3524, device='cuda:0') tensor(72, device='cuda:0') sink
